# Passo a Passo para o Treinamento com Chars74K

---

## 1. Preparação e Carregamento do Dataset

O Chars74K é um dataset mais complexo de carregar do que o MNIST.

* **1.1. Download e Organização:** Baixe o conjunto de dados completo. Ele é frequentemente distribuído em pastas separadas para os três domínios (**Natural**, **Synthesized** e **Handwritten**).
* **1.2. Definição das Classes (Rótulos):** O dataset contém 62 classes (10 dígitos e 52 letras - maiúsculas e minúsculas).
* **1.3. Pré-processamento Inicial:**
    * **Redimensionamento:** As imagens naturais têm tamanhos variados. Redimensione todas para um tamanho padrão que sua CNN possa aceitar (ex: $64 \times 64$ ou $128 \times 128$ pixels).
    * **Normalização:** Converta as imagens para *grayscale* (escala de cinza) para reduzir a complexidade ou mantenha-as coloridas (RGB), dependendo do seu objetivo. Normalize os valores de pixel para o intervalo $[0, 1]$ (dividindo por 255).

---

## 2. Divisão dos Dados (Treinamento, Validação e Teste)

A divisão é crucial para garantir que você avalie o modelo em dados que ele nunca viu.

* **2.1. Proporções:** A divisão mais comum e eficiente é:
    * **Treinamento:** $70\%$ a $80\%$ (usado para ajustar os pesos da CNN).
    * **Validação:** $10\%$ a $15\%$ (usado para ajustar os hiperparâmetros e decidir quando parar o treinamento).
    * **Teste:** $10\%$ a $15\%$ (usado apenas no final para uma avaliação imparcial da performance final).
* **2.2. Estratificação:** É altamente recomendado realizar a divisão de forma **estratificada**. Isso garante que a distribuição das 62 classes de caracteres seja mantida (proporcional) em cada um dos três subconjuntos.

---

## 3. Arquitetura da CNN e Configuração Inicial

* **3.1. Escolha da Arquitetura:** Comece com uma CNN simples (arquitetura *baseline*).
    * **Exemplo Básico:** `CONV -> RELU -> POOL -> CONV -> RELU -> POOL -> FLATTEN -> DENSE -> SOFTMAX`.
    * **Camadas:** Use 3 a 5 camadas convolucionais seguidas de *pooling*.
* **3.2. Hiperparâmetros Iniciais (para o treinamento *baseline*):**
    * **Função de Perda:** *Categorical Cross-Entropy* (Pois é um problema de classificação multiclasse).
    * **Otimizador:** Adam (Ponto de partida robusto).
    * **Taxa de Aprendizagem ($\alpha$):** $10^{-3}$ ou $10^{-4}$.
    * **Tamanho do Lote (*Batch Size*):** 32 ou 64.
    * **Épocas:** 10 a 20.
* **3.3. Treinamento *Baseline* e Documentação:** Treine o modelo inicial e **documente** a precisão (accuracy) e a perda (*loss*) nos conjuntos de treinamento e validação para estabelecer o ponto de partida.

---

## 4. Evolução na Metodologia e Otimização

Esta é a fase onde você melhora o desempenho e atende ao requisito de "evolução na metodologia".

### A. Aumento de Dados (*Data Augmentation*)

O Chars74K é diversificado, mas o aumento de dados melhora a generalização, especialmente com imagens naturais.

* **Implementação:** Aplique transformações aleatórias **apenas** no conjunto de **treinamento**:
    * Rotação (pequenos ângulos).
    * Zoom (leve).
    * Deslocamento de Largura/Altura.
    * Inclusão de ruído (*noise*).
* **Documentação:** Compare os resultados da *baseline* (Sem Augmentation) com a nova rodada (Com Augmentation).

### B. Teste de Arquiteturas Diferentes

* **CNN Mais Profunda:** Adicione mais camadas convolucionais e *pooling* para capturar características mais complexas.
* **Inclusão de *Dropout*:** Adicione camadas de *Dropout* ($20\%$ a $50\%$) após as camadas convolucionais e/ou densas para prevenir *overfitting* (um problema comum em conjuntos de dados com alta variabilidade como o Chars74K).

### C. Transfer Learning (Metodologia Avançada)

Se a precisão ainda estiver baixa, o *Transfer Learning* é a melhor forma de evolução.

* **Estratégia:** Utilize os pesos pré-treinados de uma rede grande e robusta (treinada em milhões de imagens, como **ImageNet**), como:
    * **VGG16 / VGG19**
    * **ResNet**
    * **MobileNet**
* **Ajuste Fino (*Fine-Tuning*):**
    1.  Carregue a arquitetura e os pesos pré-treinados (exclua a camada de saída).
    2.  Congele as primeiras camadas.
    3.  Adicione suas novas camadas densas (com 62 saídas, uma para cada classe).
    4.  Treine apenas as novas camadas.
    5.  (Opcional) Descongele as últimas camadas convolucionais do modelo base e treine novamente com uma taxa de aprendizado **muito baixa** ($\approx 10^{-5}$) para um ajuste fino.

### D. Ajuste de Hiperparâmetros

Otimize os hiperparâmetros com base nos resultados do conjunto de **validação**:

* **Taxa de Aprendizagem:** Use um *Learning Rate Scheduler* para reduzir a taxa durante o treinamento.
* **Otimizador:** Teste SGD com *Momentum* ou Adagrad.
* **Pesos de Classe:** Se o Chars74K tiver classes desbalanceadas, aplique pesos de classe para penalizar erros nas classes menos representadas.

---

## 5. Avaliação Final

* **Avaliação no Conjunto de Teste:** Após selecionar o melhor modelo (o que teve o melhor desempenho no conjunto de **validação**), avalie-o **apenas uma vez** no conjunto de **teste**.
* **Métricas:** Apresente a **Matriz de Confusão** e a **Acurácia (Accuracy)** geral do modelo final.